## Dictionary-based debiasing evaluation using RNSB and MAC

GPU information

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Thu Jun 16 21:18:54 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P0    29W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

RAM information

In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 27.3 gigabytes of available RAM

You are using a high-RAM runtime!


In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
!pip install wefe

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 7.9 MB 5.2 MB/s 


In [ ]:
# mount google drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import gensim.downloader as api

from wefe.metrics import MAC, RNSB
from wefe.datasets import load_weat
from wefe.datasets import load_bingliu
from wefe.query import Query
from wefe.word_embedding_model import WordEmbeddingModel
from gensim.models import KeyedVectors
from wefe.utils import run_queries
from wefe.utils import plot_queries_results
import plotly.express as px

Define the querries

In [ ]:
# Load the wordset
weat_wordset = load_weat()

# Define the 09 Queries:

    # Flowers vs Insects wrt Pleasant (5) and Unpleasant (5)
flowers_n_insects = Query([weat_wordset['flowers'], weat_wordset['insects']],
          [weat_wordset['pleasant_5'], weat_wordset['unpleasant_5']],
          ['Flowers', 'Insects'], ['Pleasant(5)', 'Unpleasant(5)'])

    # Instruments vs Weapons wrt Pleasant (5) and Unpleasant (5)
instruments_n_weapons =   Query([weat_wordset['instruments'], weat_wordset['weapons']],
          [weat_wordset['pleasant_5'], weat_wordset['unpleasant_5']],
          ['Instruments', 'Weapons'], ['Pleasant(5)', 'Unpleasant(5)'])

    # European american names(5) vs African american names(5)
    # wrt Pleasant (5) and Unpleasant (5)
european_american_names_5_n_african_american_names_5 =    Query([
        weat_wordset['european_american_names_5'],
        weat_wordset['african_american_names_5']
    ], [weat_wordset['pleasant_5'], weat_wordset['unpleasant_5']],
          ['European american names(5)', 'African american names(5)'],
          ['Pleasant(5)', 'Unpleasant(5)'])

    # European american names(7) vs African american names(7)
    # wrt Pleasant (5) and Unpleasant (5)
    # Query([
    #     weat_wordset['european_american_names_7'],
    #     weat_wordset['african_american_names_7']
    # ], [weat_wordset['pleasant_5'], weat_wordset['unpleasant_5']],
    #       ['European american names(7)', 'African american names(7)'],
    #       ['Pleasant(5)', 'Unpleasant(5)']),


    # Male and female names wrt Career and family
male_names_n_female_names =    Query([weat_wordset['male_names'], weat_wordset['female_names']],
          [weat_wordset['career'], weat_wordset['family']],
          ['Male names', 'Female names'], ['Career', 'Family'])

    # Math and arts wrt male and female terms
math_n_arts =    Query([weat_wordset['math'], weat_wordset['arts']],
          [weat_wordset['male_terms'], weat_wordset['female_terms']],
          ['Math', 'Arts'], ['Male terms', 'Female terms'])

    # Science and arts wrt male and female terms
science_n_arts_2 =    Query([weat_wordset['science'], weat_wordset['arts_2']],
          [weat_wordset['male_terms'], weat_wordset['female_terms']],
          ['Science', 'Arts 2'], ['Male terms', 'Female terms'])

    # Mental and Physical disease wrt Temporary and Permanent
mental_disease_n_physical_disease =    Query([weat_wordset['mental_disease'], weat_wordset['physical_disease']],
          [weat_wordset['temporary'], weat_wordset['permanent']],
          ['Mental disease', 'Physical disease'], ['Temporary', 'Permanent'])

    # Young people names and Old people names disease wrt Pleasant(9) and Unpleasant(9)
young_people_names_n_old_people_names =    Query(
        [weat_wordset['young_people_names'], weat_wordset['old_people_names']],
        [weat_wordset['pleasant_9'], weat_wordset['unpleasant_9']],
        ['Young peoples names', 'Old peoples names'],
        ['Pleasant(9)', 'Unpleasant(9)'])

queries = [flowers_n_insects, instruments_n_weapons, european_american_names_5_n_african_american_names_5,
           male_names_n_female_names, math_n_arts, science_n_arts_2, mental_disease_n_physical_disease, young_people_names_n_old_people_names]

Load the models

In [ ]:
w2v = WordEmbeddingModel(api.load('word2vec-google-news-300'),'word2vec Orginal')
glove = WordEmbeddingModel(api.load('glove-wiki-gigaword-300'),'glove Orginal') 
# glove  = WordEmbeddingModel(KeyedVectors.load_word2vec_format(
#     '/content/drive/MyDrive/Colab Notebooks/msc_ai_project/data/glove.840B.300d.txt',binary=False), 'glove Orginal')
fastText =  WordEmbeddingModel(api.load('fasttext-wiki-news-subwords-300'), 'fasttext - Orginal')
deb_w2v = WordEmbeddingModel(KeyedVectors.load_word2vec_format(
    '/content/drive/MyDrive/Colab Notebooks/msc_ai_project/data/word2vec.bin',binary=True), 'word2vec Debiased')
deb_glove = WordEmbeddingModel(KeyedVectors.load_word2vec_format(
    '/content/drive/MyDrive/Colab Notebooks/msc_ai_project/data/glove.bin',binary=True), 'glove Debiased')
deb_fastText = WordEmbeddingModel(KeyedVectors.load_word2vec_format(
    '/content/drive/MyDrive/Colab Notebooks/msc_ai_project/data/fasttext.bin',binary=True), 'fasttext Debiased')

models = [w2v, deb_w2v, glove, deb_glove, fastText, deb_fastText]

[==================================================] 100.0% 1662.8/1662.8MB downloaded
[==================================================] 100.0% 376.1/376.1MB downloaded
[==================================================] 100.0% 958.5/958.4MB downloaded


## Run the queries using MAC


We use  `run_queries` with `secondary_preprocessor_args : {'lowercase': True}` in `metric_params` so that when transforming the words to embeddings, first look for the original word and if you do not find it, look for the word in lower case. This feature is particularly useful for glove, since it is uncased.

In practice, it transforms the original words into word2vec and transforms the lower words into glove.

In [ ]:
wefe_results = run_queries(MAC,
                      queries,
                      models,
                      metric_params={
                          'secondary_preprocessor_args': {
                              'lowercase': True
                          },
                      }).T.round(2)

wefe_results

ERROR:root:At least one set of 'European american names(5) and African american names(5) wrt Pleasant(5) and Unpleasant(5)' query has proportionally fewer embeddings than allowed by the lost_vocabulary_threshold parameter (0.2). This query will return np.nan.
ERROR:root:At least one set of 'Male names and Female names wrt Career and Family' query has proportionally fewer embeddings than allowed by the lost_vocabulary_threshold parameter (0.2). This query will return np.nan.
ERROR:root:At least one set of 'Science and Arts 2 wrt Male terms and Female terms' query has proportionally fewer embeddings than allowed by the lost_vocabulary_threshold parameter (0.2). This query will return np.nan.
ERROR:root:At least one set of 'Young peoples names and Old peoples names wrt Pleasant(9) and Unpleasant(9)' query has proportionally fewer embeddings than allowed by the lost_vocabulary_threshold parameter (0.2). This query will return np.nan.


model_name,word2vec Orginal,word2vec Debiased,glove Orginal,glove Debiased,fasttext - Orginal,fasttext Debiased
query_name,,,,,,
Flowers and Insects wrt Pleasant(5) and Unpleasant(5),0.91,0.99,1.00,0.94,0.74,0.97
Instruments and Weapons wrt Pleasant(5) and Unpleasant(5),0.92,0.99,0.98,0.93,0.74,0.97
European american names(5) and African american names(5) wrt Pleasant(5) and Unpleasant(5),0.94,0.99,NaN,0.92,0.81,0.97
Male names and Female names wrt Career and Family,0.92,0.96,NaN,0.84,0.82,0.92
Math and Arts wrt Male terms and Female terms,0.93,1.00,0.89,0.89,0.71,0.95
Science and Arts 2 wrt Male terms and Female terms,0.92,0.98,NaN,0.87,0.70,0.93
Mental disease and Physical disease wrt Temporary and Permanent,0.87,0.95,0.86,0.83,0.64,0.92
Young peoples names and Old peoples names wrt Pleasant(9) and Unpleasant(9),0.92,0.98,NaN,0.87,0.80,0.96


Detailed MAC

In [ ]:
from wefe.metrics import WEAT
# Flowers and Insects wrt Pleasant(5) and Unpleasant(5)
r = WEAT().run_query(european_american_names_5_n_african_american_names_5,glove,lost_vocabulary_threshold=0.2, warn_not_found_words=True)
r

ERROR:root:At least one set of 'European american names(5) and African american names(5) wrt Pleasant(5) and Unpleasant(5)' query has proportionally fewer embeddings than allowed by the lost_vocabulary_threshold parameter (0.2). This query will return np.nan.


{'effect_size': nan,
 'query_name': 'European american names(5) and African american names(5) wrt Pleasant(5) and Unpleasant(5)',
 'result': nan,
 'weat': nan}

Debiased models

In [ ]:
# deb_w2v = WordEmbeddingModel(KeyedVectors.load_word2vec_format(
#     '/content/drive/MyDrive/Colab Notebooks/msc_ai_project/data/word2vec.bin',binary=True), 'word2vec Debiased')
# deb_glove = WordEmbeddingModel(KeyedVectors.load_word2vec_format(
#     '/content/drive/MyDrive/Colab Notebooks/msc_ai_project/data/glove.bin',binary=True), 'glove Debiased')
# deb_fastText = WordEmbeddingModel(KeyedVectors.load_word2vec_format(
#     '/content/drive/MyDrive/Colab Notebooks/msc_ai_project/data/fasttext.bin',binary=True), 'fasttext Debiased')
# deb_models = [deb_w2v, deb_glove, deb_fastText]

In [ ]:
# deb_wefe_results = run_queries(MAC,
#                       queries,
#                       deb_models,
#                       metric_params={
#                           'secondary_preprocessor_args': {
#                               'lowercase': True
#                           },
#                           # 'warn_not_found_words': True
#                       }).T.round(2)

# deb_wefe_results

## Run the queries using RNSB

In [ ]:
wefe_results = run_queries(RNSB,
                      queries,
                      models,
                      # n_iterations=20,
                      # holdout=True,
                      # print_model_evaluation=True,
                      metric_params={
                          'secondary_preprocessor_args': {
                              'lowercase': True
                          },
                          # 'warn_not_found_words': True
                      }).T.round(2)

wefe_results

ERROR:root:At least one set of 'European american names(5) and African american names(5) wrt Pleasant(5) and Unpleasant(5)' query has proportionally fewer embeddings than allowed by the lost_vocabulary_threshold parameter (0.2). This query will return np.nan.
ERROR:root:At least one set of 'Male names and Female names wrt Career and Family' query has proportionally fewer embeddings than allowed by the lost_vocabulary_threshold parameter (0.2). This query will return np.nan.
ERROR:root:At least one set of 'Science and Arts 2 wrt Male terms and Female terms' query has proportionally fewer embeddings than allowed by the lost_vocabulary_threshold parameter (0.2). This query will return np.nan.
ERROR:root:At least one set of 'Young peoples names and Old peoples names wrt Pleasant(9) and Unpleasant(9)' query has proportionally fewer embeddings than allowed by the lost_vocabulary_threshold parameter (0.2). This query will return np.nan.


model_name,word2vec Orginal,word2vec Debiased,glove Orginal,glove Debiased,fasttext - Orginal,fasttext Debiased
query_name,,,,,,
Flowers and Insects wrt Pleasant(5) and Unpleasant(5),0.13,0.03,0.13,0.11,0.01,0.02
Instruments and Weapons wrt Pleasant(5) and Unpleasant(5),0.14,0.05,0.33,0.09,0.02,0.02
European american names(5) and African american names(5) wrt Pleasant(5) and Unpleasant(5),0.02,0.01,NaN,0.03,0.00,0.01
Male names and Female names wrt Career and Family,0.03,0.01,NaN,0.02,0.00,0.00
Math and Arts wrt Male terms and Female terms,0.01,0.00,0.03,0.01,0.00,0.00
Science and Arts 2 wrt Male terms and Female terms,0.01,0.00,NaN,0.02,0.00,0.00
Mental disease and Physical disease wrt Temporary and Permanent,0.02,0.00,0.06,0.01,0.00,0.00
Young peoples names and Old peoples names wrt Pleasant(9) and Unpleasant(9),0.01,0.00,NaN,0.01,0.00,0.00


In [ ]:
# wefe_results = run_queries(RNSB,
#                       queries,
#                       deb_models,
#                       metric_params={
#                           'secondary_preprocessor_args': {
#                               'lowercase': True
#                           },
#                           # 'warn_not_found_words': True
#                       }).T.round(2)

# wefe_results

In [ ]:
# Load the wordset
weat_wordset = load_weat()
bing_liu = load_bingliu()

# Define the 09 Queries:

    # Flowers vs Insects wrt Pleasant (5) and Unpleasant (5)
rnsb_flowers_n_insects =    Query([weat_wordset['flowers'], weat_wordset['insects']],
          # [weat_wordset['pleasant_5'], weat_wordset['unpleasant_5']],
          [bing_liu["positive_words"], bing_liu["negative_words"]],
          ['Flowers', 'Insects'], ['Positive words', 'Negative words'])

    # Instruments vs Weapons wrt Pleasant (5) and Unpleasant (5)
rnsb_instruments_n_weapons =    Query([weat_wordset['instruments'], weat_wordset['weapons']],
          # [weat_wordset['pleasant_5'], weat_wordset['unpleasant_5']],
          [bing_liu["positive_words"], bing_liu["negative_words"]],
          ['Instruments', 'Weapons'], ['Positive words', 'Negative words'])

    # European american names(5) vs African american names(5)
    # wrt Pleasant (5) and Unpleasant (5)
rnsb_european_american_names_5_n_african_american_names_5 =    Query([
        weat_wordset['european_american_names_5'],
        weat_wordset['african_american_names_5']], 
        # [weat_wordset['pleasant_5'], weat_wordset['unpleasant_5']],
        [bing_liu["positive_words"], bing_liu["negative_words"]],
          ['European american names(5)', 'African american names(5)'],
          ['Positive words', 'Negative words'])

    # European american names(7) vs African american names(7)
    # wrt Pleasant (5) and Unpleasant (5)
    # Query([
    #     weat_wordset['european_american_names_7'],
    #     weat_wordset['african_american_names_7']
    # ], [weat_wordset['pleasant_5'], weat_wordset['unpleasant_5']],
    #       ['European american names(7)', 'African american names(7)'],
    #       ['Pleasant(5)', 'Unpleasant(5)']),


    # Male and female names wrt Career and family
rnsb_male_names_n_female_names =    Query([weat_wordset['male_names'], weat_wordset['female_names']],
          # [weat_wordset['career'], weat_wordset['family']],
          [bing_liu["positive_words"], bing_liu["negative_words"]],
          ['Male names', 'Female names'], ['Positive words', 'Negative words'])

    # Math and arts wrt male and female terms
rnsb_math_n_arts =    Query([weat_wordset['math'], weat_wordset['arts']],
          # [weat_wordset['male_terms'], weat_wordset['female_terms']],
          [bing_liu["positive_words"], bing_liu["negative_words"]],
          ['Math', 'Arts'], ['Positive words', 'Negative words'])

    # Science and arts wrt male and female terms
rnsb_science_n_arts_2 =    Query([weat_wordset['science'], weat_wordset['arts_2']],
          # [weat_wordset['male_terms'], weat_wordset['female_terms']],
          [bing_liu["positive_words"], bing_liu["negative_words"]],
          ['Science', 'Arts 2'], ['Positive words', 'Negative words'])

    # Mental and Physical disease wrt Temporary and Permanent
rnsb_mental_disease_n_physical_disease =    Query([weat_wordset['mental_disease'], weat_wordset['physical_disease']],
          # [weat_wordset['temporary'], weat_wordset['permanent']],
          [bing_liu["positive_words"], bing_liu["negative_words"]],
          ['Mental disease', 'Physical disease'], ['Positive words', 'Negative words'])

    # Young people names and Old people names disease wrt Pleasant(9) and Unpleasant(9)
rnsb_young_people_names_n_old_people_names =    Query(
        [weat_wordset['young_people_names'], weat_wordset['old_people_names']],
        # [weat_wordset['pleasant_9'], weat_wordset['unpleasant_9']],
        [bing_liu["positive_words"], bing_liu["negative_words"]],
        ['Young peoples names', 'Old peoples names'],
        ['Positive words', 'Negative words'])


rnsb_queries = [rnsb_flowers_n_insects, rnsb_instruments_n_weapons, 
                rnsb_european_american_names_5_n_african_american_names_5,
           rnsb_male_names_n_female_names, rnsb_math_n_arts, rnsb_science_n_arts_2, 
           rnsb_mental_disease_n_physical_disease, rnsb_young_people_names_n_old_people_names]

original models

In [ ]:
orig_results = run_queries(RNSB,
                      rnsb_queries,
                      models,
                      # n_iterations=20,
                      # holdout=True,
                      # print_model_evaluation=True,
                      metric_params={
                          'secondary_preprocessor_args': {
                              'lowercase': True
                          },
                          # 'warn_not_found_words': True
                      }).T.round(2)

orig_results

ERROR:root:At least one set of 'European american names(5) and African american names(5) wrt Positive words and Negative words' query has proportionally fewer embeddings than allowed by the lost_vocabulary_threshold parameter (0.2). This query will return np.nan.
ERROR:root:At least one set of 'Male names and Female names wrt Positive words and Negative words' query has proportionally fewer embeddings than allowed by the lost_vocabulary_threshold parameter (0.2). This query will return np.nan.
ERROR:root:At least one set of 'Science and Arts 2 wrt Positive words and Negative words' query has proportionally fewer embeddings than allowed by the lost_vocabulary_threshold parameter (0.2). This query will return np.nan.
ERROR:root:At least one set of 'Young peoples names and Old peoples names wrt Positive words and Negative words' query has proportionally fewer embeddings than allowed by the lost_vocabulary_threshold parameter (0.2). This query will return np.nan.


model_name,word2vec Orginal,word2vec Debiased,glove Orginal,glove Debiased,fasttext - Orginal,fasttext Debiased
query_name,,,,,,
Flowers and Insects wrt Positive words and Negative words,0.27,0.20,0.15,0.21,0.14,0.16
Instruments and Weapons wrt Positive words and Negative words,0.21,0.16,0.28,0.21,0.09,0.07
European american names(5) and African american names(5) wrt Positive words and Negative words,0.26,0.11,NaN,0.33,0.05,0.21
Male names and Female names wrt Positive words and Negative words,0.15,0.06,NaN,0.21,0.01,0.10
Math and Arts wrt Positive words and Negative words,0.51,0.31,0.79,0.30,0.20,0.15
Science and Arts 2 wrt Positive words and Negative words,0.90,0.23,NaN,0.61,0.17,0.14
Mental disease and Physical disease wrt Positive words and Negative words,0.01,0.02,0.05,0.01,0.00,0.00
Young peoples names and Old peoples names wrt Positive words and Negative words,0.21,0.10,NaN,0.22,0.03,0.11


## rnsb_flowers_n_insects


### w2v

In [ ]:
orig_w2v_results = RNSB().run_query(
        rnsb_flowers_n_insects, w2v, holdout=True, print_model_evaluation=True, n_iterations=1000, 
    )
orig_w2v_results

Classification Report:
              precision    recall  f1-score   support

        -1.0       0.96      0.97      0.97       889
         1.0       0.93      0.91      0.92       372

    accuracy                           0.95      1261
   macro avg       0.95      0.94      0.94      1261
weighted avg       0.95      0.95      0.95      1261

When n_iterations > 1, only the first evaluation is printed.


{'negative_sentiment_distribution': {'ant': 0.03151620535339805,
  'aster': 0.005349036235697446,
  'azalea': 0.00522924764824657,
  'bedbug': 0.03263580664780354,
  'bee': 0.01718326661564784,
  'beetle': 0.03155690880121419,
  'blackfly': 0.029676649751039976,
  'bluebell': 0.0007279376904486328,
  'buttercup': 0.018233021133619867,
  'carnation': 0.00548151962654912,
  'caterpillar': 0.031215056190689355,
  'centipede': 0.03178544407579192,
  'clover': 0.013048616125779698,
  'cockroach': 0.032350825760362745,
  'cricket': 0.023106087025844563,
  'crocus': 0.025170587392499513,
  'daffodil': 0.011861253534760172,
  'daisy': 0.0031858449530100543,
  'dragonfly': 0.009666928628702406,
  'flea': 0.032158181128746445,
  'fly': 0.024529733161574727,
  'gladiola': 0.013727821445383318,
  'gnat': 0.0327163659532421,
  'hornet': 0.03255064633119552,
  'horsefly': 0.03227508358180748,
  'hyacinth': 0.025899026125808133,
  'iris': 0.0008866725839790692,
  'lilac': 0.002426390751474046,
  'lil

In [ ]:
# query = Query(flowers_n_insects, [bing_liu["positive_words"], bing_liu["negative_words"]])

deb_w2v_results = RNSB().run_query(
        rnsb_flowers_n_insects, deb_w2v, holdout=True, print_model_evaluation=True, n_iterations=1000, 
    )
deb_w2v_results
# fig = px.bar(
#         pd.DataFrame(
#             results["negative_sentiment_distribution"].items(),
#             columns=["Word", "Sentiment distribution"],
#         ),
#         x="Word",
#         y="Sentiment distribution",
#         title=f"Word2Vec Negative Sentiment Distribution",
#     )

# fig.update_yaxes(range=[0, 0.2])
# fig.show()

Classification Report:
              precision    recall  f1-score   support

        -1.0       0.95      0.98      0.97       889
         1.0       0.95      0.89      0.92       372

    accuracy                           0.95      1261
   macro avg       0.95      0.93      0.94      1261
weighted avg       0.95      0.95      0.95      1261

When n_iterations > 1, only the first evaluation is printed.


{'negative_sentiment_distribution': {'ant': 0.029485501011813443,
  'aster': 0.00906867212230954,
  'azalea': 0.006890036618932175,
  'bedbug': 0.03141637135503529,
  'bee': 0.019279594002919974,
  'beetle': 0.028333178661926056,
  'blackfly': 0.027065805961995433,
  'bluebell': 0.0017717461852362704,
  'buttercup': 0.01646017758287172,
  'carnation': 0.01030045961541706,
  'caterpillar': 0.028278393321810605,
  'centipede': 0.030112034829543985,
  'clover': 0.015108231990283171,
  'cockroach': 0.030898194302774233,
  'cricket': 0.024509365828356364,
  'crocus': 0.021725848083580107,
  'daffodil': 0.01573882886467703,
  'daisy': 0.004061479528259544,
  'dragonfly': 0.012185977658855526,
  'flea': 0.030739351643002677,
  'fly': 0.02508245610724608,
  'gladiola': 0.013565671734809808,
  'gnat': 0.03182313611347929,
  'hornet': 0.0314791406586078,
  'horsefly': 0.03076379361381548,
  'hyacinth': 0.021378033588581927,
  'iris': 0.0021965897675060484,
  'lilac': 0.004473456143456077,
  'lil

fasttext

In [ ]:
orig_fastText_results = RNSB().run_query(
        rnsb_flowers_n_insects, fastText, holdout=True, print_model_evaluation=True, n_iterations=1000, 
    )
orig_fastText_results

Classification Report:
              precision    recall  f1-score   support

        -1.0       0.95      0.98      0.96       916
         1.0       0.94      0.88      0.91       389

    accuracy                           0.95      1305
   macro avg       0.95      0.93      0.94      1305
weighted avg       0.95      0.95      0.95      1305

When n_iterations > 1, only the first evaluation is printed.


{'negative_sentiment_distribution': {'ant': 0.031249732302154398,
  'aster': 0.01217032798641653,
  'azalea': 0.008496867720860354,
  'bedbug': 0.032625699016975866,
  'bee': 0.025791683156722506,
  'beetle': 0.028947413816996052,
  'blackfly': 0.030107426914140437,
  'bluebell': 0.014899165831303648,
  'buttercup': 0.016794569689253587,
  'carnation': 0.012541972787168636,
  'caterpillar': 0.025444923548916563,
  'centipede': 0.028691485382256236,
  'clover': 0.009445023811431254,
  'cockroach': 0.02982972825524206,
  'cricket': 0.008343993280207355,
  'crocus': 0.022603282450500573,
  'daffodil': 0.008424932946860824,
  'daisy': 0.014867165806433017,
  'dragonfly': 0.007754033698129853,
  'flea': 0.031136661519719243,
  'fly': 0.02080663846965795,
  'gnat': 0.03291629009260238,
  'hornet': 0.030278356268513322,
  'horsefly': 0.02957743022173362,
  'hyacinth': 0.02359114441555896,
  'iris': 0.008671428095330695,
  'lilac': 0.004231197698340012,
  'lily': 0.011547216920265377,
  'locus

In [ ]:
deb_fastText_results = RNSB().run_query(
        rnsb_flowers_n_insects, deb_fastText, holdout=True, print_model_evaluation=True, n_iterations=1000, 
    )
deb_fastText_results

Classification Report:
              precision    recall  f1-score   support

        -1.0       0.96      0.99      0.97       916
         1.0       0.96      0.89      0.93       389

    accuracy                           0.96      1305
   macro avg       0.96      0.94      0.95      1305
weighted avg       0.96      0.96      0.96      1305

When n_iterations > 1, only the first evaluation is printed.


{'negative_sentiment_distribution': {'ant': 0.030132025895408504,
  'aster': 0.025203230231518037,
  'azalea': 0.013185899729258805,
  'bedbug': 0.03191265265887628,
  'bee': 0.021285987627148915,
  'beetle': 0.02730479889132249,
  'blackfly': 0.02851872125392953,
  'bluebell': 0.017549043511389594,
  'buttercup': 0.01219190223114013,
  'carnation': 0.008649290255476528,
  'caterpillar': 0.01891491201094054,
  'centipede': 0.03182738203240044,
  'clover': 0.0041415300848433685,
  'cockroach': 0.03093547733029004,
  'cricket': 0.015457918228553894,
  'crocus': 0.021200670590621522,
  'daffodil': 0.010962979505751045,
  'daisy': 0.02402891334856733,
  'dragonfly': 0.0012058096404803991,
  'flea': 0.028444010189788285,
  'fly': 0.021509378922212965,
  'gnat': 0.03214722323036263,
  'hornet': 0.028463471207631173,
  'horsefly': 0.03133908315950745,
  'hyacinth': 0.019929238704324403,
  'iris': 0.013672956038912738,
  'lilac': 0.004107230462883355,
  'lily': 0.012338487760071267,
  'locust'

 GloVE

In [ ]:
orig_glove_results = RNSB().run_query(
        rnsb_flowers_n_insects, glove, holdout=True, print_model_evaluation=True, n_iterations=1000, 
    )
orig_glove_results

Classification Report:
              precision    recall  f1-score   support

        -1.0       0.92      0.94      0.93       869
         1.0       0.86      0.81      0.83       379

    accuracy                           0.90      1248
   macro avg       0.89      0.88      0.88      1248
weighted avg       0.90      0.90      0.90      1248

When n_iterations > 1, only the first evaluation is printed.


{'negative_sentiment_distribution': {'ant': 0.029385245781202638,
  'aster': 0.01151844714741534,
  'azalea': 0.012236146901895565,
  'bedbug': 0.029300172645683218,
  'bee': 0.02628809659910992,
  'beetle': 0.02031029601409757,
  'blackfly': 0.021624005131284863,
  'bluebell': 0.003672023684975748,
  'buttercup': 0.02204645302723436,
  'carnation': 0.012021077194817166,
  'caterpillar': 0.029344265683641353,
  'centipede': 0.02930663393175745,
  'clover': 0.010028471156147994,
  'cockroach': 0.02697241591317778,
  'cricket': 0.0052985331323277506,
  'crocus': 0.01679284603454406,
  'daffodil': 0.011853121977889123,
  'daisy': 0.019335195189354754,
  'dragonfly': 0.003419261252501071,
  'flea': 0.015276196911920763,
  'fly': 0.01921985029433418,
  'gnat': 0.02895642467531878,
  'hornet': 0.027177543563133302,
  'horsefly': 0.02856776385479745,
  'hyacinth': 0.019101770425068682,
  'iris': 0.022610007614303356,
  'lilac': 0.008813179658685248,
  'lily': 0.019711800979586747,
  'locust':

In [ ]:
deb_glove_results = RNSB().run_query(
        rnsb_flowers_n_insects, deb_glove, holdout=True, print_model_evaluation=True, n_iterations=1000, 
    )
deb_glove_results

Classification Report:
              precision    recall  f1-score   support

        -1.0       0.97      0.98      0.97       929
         1.0       0.95      0.93      0.94       395

    accuracy                           0.96      1324
   macro avg       0.96      0.95      0.96      1324
weighted avg       0.96      0.96      0.96      1324

When n_iterations > 1, only the first evaluation is printed.


{'negative_sentiment_distribution': {'ant': 0.03086756941773818,
  'aster': 0.015764791137155367,
  'azalea': 0.020110585678936622,
  'bedbug': 0.03173212387905474,
  'bee': 0.011701664780913916,
  'beetle': 0.030505623279167778,
  'blackfly': 0.031634051729867795,
  'bluebell': 0.0022467804881077004,
  'buttercup': 0.015109482313866279,
  'carnation': 0.006317962607276622,
  'caterpillar': 0.031008125038245835,
  'centipede': 0.031907679102279915,
  'clover': 0.003075756107671593,
  'cockroach': 0.03131452997288875,
  'cricket': 0.0008083075372346578,
  'crocus': 0.013151890145421728,
  'daffodil': 0.010394631570778624,
  'daisy': 0.00915998440035309,
  'dragonfly': 0.004732739880528857,
  'flea': 0.023693123321589835,
  'fly': 0.023512382880407554,
  'gladiola': 0.019799411853694743,
  'gnat': 0.03205778390598542,
  'hornet': 0.031803185392171314,
  'horsefly': 0.03124167900713254,
  'hyacinth': 0.019474374180737927,
  'iris': 0.0033550727748873697,
  'lilac': 0.006764322877938622,
 

## rnsb_instruments_n_weapons,                 

In [ ]:
orig_w2v_results = RNSB().run_query(
        rnsb_instruments_n_weapons, w2v, holdout=True, print_model_evaluation=True, n_iterations=1000, 
    )
orig_w2v_results

Classification Report:
              precision    recall  f1-score   support

        -1.0       0.97      0.96      0.97       889
         1.0       0.91      0.93      0.92       372

    accuracy                           0.95      1261
   macro avg       0.94      0.95      0.94      1261
weighted avg       0.95      0.95      0.95      1261

When n_iterations > 1, only the first evaluation is printed.


{'negative_sentiment_distribution': {'arrow': 0.03148401559724218,
  'bagpipe': 0.01568719905220316,
  'banjo': 0.01750964798781707,
  'bassoon': 0.020768076312263874,
  'bell': 0.023013626270052507,
  'blade': 0.03340267900805288,
  'bomb': 0.03355940471671917,
  'bongo': 0.02699003437212197,
  'cannon': 0.017457414778563118,
  'cello': 0.001305643626214427,
  'clarinet': 0.004270490055315037,
  'club': 0.012797212054977102,
  'dagger': 0.030528409728676188,
  'drum': 0.029647153249716597,
  'dynamite': 0.032290916941303316,
  'fiddle': 0.026155548769478572,
  'firearm': 0.03339230352613031,
  'flute': 0.0010134472314448774,
  'grenade': 0.03321550521210188,
  'guitar': 0.010029916993155693,
  'gun': 0.03345808164355377,
  'harmonica': 0.005493482239919577,
  'harp': 0.008011756564870503,
  'harpoon': 0.033334521876939825,
  'harpsichord': 0.00023484884460781485,
  'hatchet': 0.03323681854189191,
  'horn': 0.022744806285210944,
  'knife': 0.033562336112665445,
  'lute': 0.013272459998

In [ ]:

deb_w2v_results = RNSB().run_query(
        rnsb_instruments_n_weapons, deb_w2v, holdout=True, print_model_evaluation=True, n_iterations=1000, 
    )
deb_w2v_results

Classification Report:
              precision    recall  f1-score   support

        -1.0       0.96      0.97      0.96       889
         1.0       0.93      0.90      0.91       372

    accuracy                           0.95      1261
   macro avg       0.94      0.94      0.94      1261
weighted avg       0.95      0.95      0.95      1261

When n_iterations > 1, only the first evaluation is printed.


{'negative_sentiment_distribution': {'arrow': 0.028657839059201374,
  'bagpipe': 0.017697459371938985,
  'banjo': 0.01830831471930232,
  'bassoon': 0.021768050169613883,
  'bell': 0.0207960949154717,
  'blade': 0.03140460746931046,
  'bomb': 0.03213121645935216,
  'bongo': 0.02370499060099242,
  'cannon': 0.01890867329441272,
  'cello': 0.003380934777918291,
  'clarinet': 0.007231969966501927,
  'club': 0.015818331423526052,
  'dagger': 0.02905132353165105,
  'drum': 0.027864368836127112,
  'dynamite': 0.0289149381579411,
  'fiddle': 0.024294138092399765,
  'firearm': 0.03147335866587836,
  'flute': 0.0027435601718287924,
  'grenade': 0.03119265308520438,
  'guitar': 0.013028122237868119,
  'gun': 0.031698389148418465,
  'harmonica': 0.009303777527833087,
  'harp': 0.014125043212013362,
  'harpoon': 0.03170485222146184,
  'harpsichord': 0.001100456444556637,
  'hatchet': 0.031008524129837588,
  'horn': 0.0218312922839179,
  'knife': 0.03205950151056092,
  'lute': 0.01575760957101066,
 

In [ ]:
orig_fastText_results = RNSB().run_query(
        rnsb_instruments_n_weapons, fastText, holdout=True, print_model_evaluation=True, n_iterations=1000, 
    )
orig_fastText_results

Classification Report:
              precision    recall  f1-score   support

        -1.0       0.96      0.98      0.97       916
         1.0       0.95      0.89      0.92       389

    accuracy                           0.96      1305
   macro avg       0.95      0.94      0.95      1305
weighted avg       0.96      0.96      0.96      1305

When n_iterations > 1, only the first evaluation is printed.


{'negative_sentiment_distribution': {'arrow': 0.02405941433327476,
  'axe': 0.02979149015969035,
  'bagpipe': 0.018871704775615977,
  'banjo': 0.01417698994194318,
  'bassoon': 0.015358042428438856,
  'bell': 0.023661499029511018,
  'blade': 0.025443332810284318,
  'bomb': 0.02943794203983281,
  'bongo': 0.023317414517772864,
  'cannon': 0.02303841947335037,
  'cello': 0.008748672256518222,
  'clarinet': 0.009284685503733505,
  'club': 0.0034146390774526247,
  'dagger': 0.02732003180668955,
  'drum': 0.020658966853727388,
  'dynamite': 0.02285164407812238,
  'fiddle': 0.022729262747066437,
  'firearm': 0.027053954381546026,
  'flute': 0.006698253800253323,
  'grenade': 0.029167079343484885,
  'guitar': 0.00878595326502787,
  'gun': 0.029281797083791136,
  'harmonica': 0.013341208044504468,
  'harp': 0.024703202065960484,
  'harpoon': 0.024196229212128318,
  'harpsichord': 0.011041018929346136,
  'hatchet': 0.02878188861670897,
  'horn': 0.027493071492076415,
  'knife': 0.02859408180399

In [ ]:
deb_fastText_results = RNSB().run_query(
        rnsb_instruments_n_weapons, deb_fastText, holdout=True, print_model_evaluation=True, n_iterations=1000, 
    )
deb_fastText_results

Classification Report:
              precision    recall  f1-score   support

        -1.0       0.97      0.98      0.97       916
         1.0       0.95      0.92      0.93       389

    accuracy                           0.96      1305
   macro avg       0.96      0.95      0.95      1305
weighted avg       0.96      0.96      0.96      1305

When n_iterations > 1, only the first evaluation is printed.


{'negative_sentiment_distribution': {'arrow': 0.019906228849265652,
  'axe': 0.026440848162302355,
  'bagpipe': 0.021455586251547115,
  'banjo': 0.022254989790260072,
  'bassoon': 0.02156441475240168,
  'bell': 0.019163505799527403,
  'blade': 0.02357709492847306,
  'bomb': 0.025764878313764936,
  'bongo': 0.02477489744859193,
  'cannon': 0.02218882850551176,
  'cello': 0.01962605904468315,
  'clarinet': 0.020408513586934268,
  'club': 0.006883176571409609,
  'dagger': 0.023791907708169545,
  'drum': 0.02323116012803511,
  'dynamite': 0.023949507116016355,
  'fiddle': 0.02482124234976813,
  'firearm': 0.025791602900389496,
  'flute': 0.006278773970905497,
  'grenade': 0.026481575142256714,
  'guitar': 0.011985408509430969,
  'gun': 0.025801900537326394,
  'harmonica': 0.016329891756394772,
  'harp': 0.01516679612259907,
  'harpoon': 0.026376927127164346,
  'harpsichord': 0.009778206303802993,
  'hatchet': 0.025468928995522354,
  'horn': 0.02504789550621672,
  'knife': 0.025939909344921

In [ ]:
glove_instrument_results = RNSB().run_query(
        rnsb_instruments_n_weapons, glove, holdout=True, print_model_evaluation=True, n_iterations=1000, 
    )
glove_instrument_results

Classification Report:
              precision    recall  f1-score   support

        -1.0       0.94      0.94      0.94       869
         1.0       0.86      0.86      0.86       379

    accuracy                           0.91      1248
   macro avg       0.90      0.90      0.90      1248
weighted avg       0.91      0.91      0.91      1248

When n_iterations > 1, only the first evaluation is printed.


{'negative_sentiment_distribution': {'arrow': 0.010553565160481174,
  'axe': 0.03322397941742516,
  'bagpipe': 0.031092383469052184,
  'banjo': 0.013488706690760294,
  'bassoon': 0.028827395703104223,
  'bell': 0.01750170861849552,
  'blade': 0.03202293714978052,
  'bomb': 0.03327423848912142,
  'bongo': 0.023247650343601576,
  'cannon': 0.02468453322487813,
  'cello': 0.004000950988266034,
  'clarinet': 0.00549338757284477,
  'club': 0.00011271404161705991,
  'dagger': 0.025504025844255682,
  'drum': 0.03153411865045511,
  'dynamite': 0.024087222584280296,
  'fiddle': 0.009954577679441247,
  'firearm': 0.03159931406479071,
  'flute': 0.0036626557212341153,
  'grenade': 0.033293181352491825,
  'guitar': 0.0014439318123162504,
  'gun': 0.02455611625331325,
  'harmonica': 0.008048961961474544,
  'harp': 0.011155167421738452,
  'harpoon': 0.020932704397915603,
  'harpsichord': 0.0017969650253282395,
  'hatchet': 0.03274904838877521,
  'horn': 0.03261134810261538,
  'knife': 0.033195662021

In [ ]:
deb_glove_instrument_results = RNSB().run_query(
        rnsb_instruments_n_weapons, deb_glove, holdout=True, print_model_evaluation=True, n_iterations=1000, 
    )
deb_glove_instrument_results

Classification Report:
              precision    recall  f1-score   support

        -1.0       0.96      0.98      0.97       929
         1.0       0.94      0.92      0.93       395

    accuracy                           0.96      1324
   macro avg       0.95      0.95      0.95      1324
weighted avg       0.96      0.96      0.96      1324

When n_iterations > 1, only the first evaluation is printed.


{'negative_sentiment_distribution': {'arrow': 0.015884175271247712,
  'axe': 0.02870524423610258,
  'bagpipe': 0.029866896095030824,
  'banjo': 0.01120757197671754,
  'bassoon': 0.021105854481973035,
  'bell': 0.0277570899259785,
  'blade': 0.03042160236051818,
  'bomb': 0.03330407239347228,
  'bongo': 0.028931637598272573,
  'cannon': 0.028740841542868482,
  'cello': 0.0027252978574696268,
  'clarinet': 0.004623701213795227,
  'club': 0.015006500686666358,
  'dagger': 0.032314968750391714,
  'drum': 0.030962281188774053,
  'dynamite': 0.018012094152358132,
  'fiddle': 0.02263509975491859,
  'firearm': 0.030927147029978938,
  'flute': 0.006458571800664139,
  'grenade': 0.033385451602967395,
  'guitar': 0.0012587596969090582,
  'gun': 0.03257642660627624,
  'harmonica': 0.005238478023369783,
  'harp': 0.008019548634707068,
  'harpoon': 0.031308951207848226,
  'harpsichord': 0.0033985942808788493,
  'hatchet': 0.032462193463151566,
  'horn': 0.021944827725858138,
  'knife': 0.03296850908

## rnsb_european_american_names_5_n_african_american_names_5,
           

In [ ]:
orig_w2v_results = RNSB().run_query(
        rnsb_european_american_names_5_n_african_american_names_5, w2v, holdout=True, print_model_evaluation=True, n_iterations=1000, 
    )
orig_w2v_results

{'negative_sentiment_distribution': {'Adam': 0.012779920834955067,
  'Alan': 0.008146112313173267,
  'Alonzo': 0.013513463574445675,
  'Alphonse': 0.029829152292021995,
  'Amanda': 0.0048323960577005675,
  'Amber': 0.0098678013608634,
  'Andrew': 0.008013540236945702,
  'Betsy': 0.004869501345644123,
  'Brad': 0.012578372705743676,
  'Colleen': 0.008111279998169232,
  'Courtney': 0.022331327170891056,
  'Darnell': 0.03097185155166107,
  'Deion': 0.02734751460071069,
  'Ebony': 0.014290993488655267,
  'Ellen': 0.0013341393262895729,
  'Emily': 0.003678183723236671,
  'Frank': 0.0063234005356417235,
  'Greg': 0.007255673707241405,
  'Harry': 0.022537908158007283,
  'Heather': 0.005050149430050523,
  'Jack': 0.016086394265473584,
  'Jamel': 0.03220320545564179,
  'Jasmine': 0.007143586571556714,
  'Jerome': 0.0263701098004106,
  'Jonathan': 0.01568773754662907,
  'Josh': 0.008711194640741875,
  'Justin': 0.006769940644577795,
  'Katie': 0.0067198886413917126,
  'Kristin': 0.00399250278592

In [ ]:
deb_w2v_results = RNSB().run_query(
        rnsb_european_american_names_5_n_african_american_names_5, deb_w2v, holdout=True, print_model_evaluation=True, n_iterations=1000, 
    )
deb_w2v_results

Classification Report:
              precision    recall  f1-score   support

        -1.0       0.96      0.97      0.97       889
         1.0       0.93      0.91      0.92       372

    accuracy                           0.95      1261
   macro avg       0.94      0.94      0.94      1261
weighted avg       0.95      0.95      0.95      1261

When n_iterations > 1, only the first evaluation is printed.


{'negative_sentiment_distribution': {'Adam': 0.01352971781997125,
  'Alan': 0.010240548664944551,
  'Alonzo': 0.014024187714611855,
  'Alphonse': 0.024717540024440612,
  'Amanda': 0.00841858353555747,
  'Amber': 0.012204968505505528,
  'Andrew': 0.010408207239719296,
  'Betsy': 0.007595149682133687,
  'Brad': 0.014430256876373384,
  'Colleen': 0.01149321448842678,
  'Courtney': 0.020712192243321888,
  'Darnell': 0.026637873782151846,
  'Deion': 0.022825184139576498,
  'Ebony': 0.014142482793025825,
  'Ellen': 0.003719953086639213,
  'Emily': 0.00726651674002786,
  'Frank': 0.010296083034029147,
  'Greg': 0.009447285004922099,
  'Harry': 0.020599843055968608,
  'Heather': 0.008853501334404629,
  'Jack': 0.015601424618377813,
  'Jamel': 0.02685596719335595,
  'Jasmine': 0.010373914932273784,
  'Jerome': 0.02199164259217428,
  'Jonathan': 0.014011420811713507,
  'Josh': 0.0093007251113764,
  'Justin': 0.00872322325745651,
  'Katie': 0.01116624169330995,
  'Kristin': 0.00721130343579777,
 

In [ ]:
orig_fastText_results = RNSB().run_query(
        rnsb_european_american_names_5_n_african_american_names_5, fastText, holdout=True, print_model_evaluation=True, n_iterations=1000, 
    )
orig_fastText_results

Classification Report:
              precision    recall  f1-score   support

        -1.0       0.96      0.98      0.97       916
         1.0       0.94      0.90      0.92       389

    accuracy                           0.95      1305
   macro avg       0.95      0.94      0.95      1305
weighted avg       0.95      0.95      0.95      1305

When n_iterations > 1, only the first evaluation is printed.


{'negative_sentiment_distribution': {'Adam': 0.017175864791728973,
  'Alan': 0.01427231666937892,
  'Alonzo': 0.022859550998282668,
  'Alphonse': 0.016909718301697716,
  'Amanda': 0.015163041153429571,
  'Amber': 0.01437027748823746,
  'Andrew': 0.015357481533816891,
  'Betsy': 0.00770126126067203,
  'Brad': 0.015477353770806977,
  'Colleen': 0.009734873759542245,
  'Courtney': 0.019304282951077223,
  'Darnell': 0.019870327920272397,
  'Deion': 0.012548794154863124,
  'Ebony': 0.011974773645002539,
  'Ellen': 0.012456299066258628,
  'Emily': 0.015706525941564745,
  'Frank': 0.019845281496549363,
  'Greg': 0.015032428755562141,
  'Harry': 0.017621624356917713,
  'Heather': 0.016049135036099272,
  'Jack': 0.018904044162596535,
  'Jamel': 0.017808597819107767,
  'Jasmine': 0.0063313708215433275,
  'Jerome': 0.018530013416007893,
  'Jonathan': 0.01511128658195851,
  'Josh': 0.012808539562745606,
  'Justin': 0.016270300063744653,
  'Katie': 0.010242929266218004,
  'Kristin': 0.0084938075936

In [ ]:
deb_fastText_results = RNSB().run_query(
        rnsb_european_american_names_5_n_african_american_names_5, deb_fastText, holdout=True, print_model_evaluation=True, n_iterations=1000, 
    )
deb_fastText_results

Classification Report:
              precision    recall  f1-score   support

        -1.0       0.96      0.98      0.97       916
         1.0       0.95      0.90      0.93       389

    accuracy                           0.96      1305
   macro avg       0.96      0.94      0.95      1305
weighted avg       0.96      0.96      0.96      1305

When n_iterations > 1, only the first evaluation is printed.


{'negative_sentiment_distribution': {'Adam': 0.013538305435961422,
  'Alan': 0.006874688817827061,
  'Alonzo': 0.03051030808879383,
  'Alphonse': 0.026291340832653224,
  'Amanda': 0.007174643954469462,
  'Amber': 0.009654157698122714,
  'Andrew': 0.011798866905048645,
  'Betsy': 0.009672950275118084,
  'Brad': 0.00408786602220828,
  'Colleen': 0.015449206333667689,
  'Courtney': 0.025452324543367055,
  'Darnell': 0.03360866702926607,
  'Deion': 0.027413600752377677,
  'Ebony': 0.009694131564399841,
  'Ellen': 0.011336651490602088,
  'Emily': 0.007762861903736242,
  'Frank': 0.008347386584987251,
  'Greg': 0.013263428933888747,
  'Harry': 0.018460127846745437,
  'Heather': 0.010186729208165539,
  'Jack': 0.012264136981372124,
  'Jamel': 0.024755434992722657,
  'Jasmine': 0.009256573301467192,
  'Jerome': 0.013789176371670403,
  'Jonathan': 0.0040836743240424194,
  'Josh': 0.00896387059295262,
  'Justin': 0.006523592411720083,
  'Katie': 0.005500210499098251,
  'Kristin': 0.0039884245190

## rnsb_male_names_n_female_names, 

In [ ]:
orig_w2v_results = RNSB().run_query(
        rnsb_male_names_n_female_names, w2v, holdout=True, print_model_evaluation=True, n_iterations=1000, 
    )
orig_w2v_results

Classification Report:
              precision    recall  f1-score   support

        -1.0       0.96      0.97      0.96       889
         1.0       0.92      0.91      0.91       372

    accuracy                           0.95      1261
   macro avg       0.94      0.94      0.94      1261
weighted avg       0.95      0.95      0.95      1261

When n_iterations > 1, only the first evaluation is printed.


{'negative_sentiment_distribution': {'Amy': 0.03720825421422022,
  'Ann': 0.05166953563414424,
  'Bill': 0.060304290188036555,
  'Diana': 0.07587977433073646,
  'Donna': 0.03168388038800149,
  'Greg': 0.06185351826930248,
  'Jeff': 0.0429450552622808,
  'Joan': 0.0616955108002039,
  'John': 0.07088186315847414,
  'Kate': 0.020440089423475832,
  'Kevin': 0.1978657936218575,
  'Lisa': 0.05513144411069061,
  'Mike': 0.10180526608098824,
  'Paul': 0.04533929006933847,
  'Sarah': 0.03650631371548251,
  'Steve': 0.048790120732766576},
 'negative_sentiment_probabilities': {'Amy': 0.12922295969365602,
  'Ann': 0.17944648201444444,
  'Bill': 0.20943468122578504,
  'Diana': 0.2635277904588453,
  'Donna': 0.11003700347630352,
  'Greg': 0.21481509592487608,
  'Jeff': 0.1491466681895181,
  'Joan': 0.2142663415357809,
  'John': 0.2461702205431114,
  'Kate': 0.07098771247093559,
  'Kevin': 0.687180949870491,
  'Lisa': 0.19146956853047095,
  'Mike': 0.35356611249870684,
  'Paul': 0.15746176156069056,


In [ ]:
w2v_fig = px.bar(
        pd.DataFrame(
            orig_w2v_results["negative_sentiment_distribution"].items(),
            columns=["Word", "Sentiment distribution"],
        ),
        x="Word",
        y="Sentiment distribution",
        title=f"Word2Vec (original) Female vs Male Names Negative Sentiment Distribution",
    )

w2v_fig.update_yaxes(range=[0, 0.2])
w2v_fig.show()

In [ ]:
deb_w2v_results = RNSB().run_query(
        rnsb_male_names_n_female_names, deb_w2v, holdout=True, print_model_evaluation=True, n_iterations=1000, 
    )
deb_w2v_results

Classification Report:
              precision    recall  f1-score   support

        -1.0       0.96      0.98      0.97       889
         1.0       0.96      0.90      0.93       372

    accuracy                           0.96      1261
   macro avg       0.96      0.94      0.95      1261
weighted avg       0.96      0.96      0.96      1261

When n_iterations > 1, only the first evaluation is printed.


{'negative_sentiment_distribution': {'Amy': 0.046803693811991745,
  'Ann': 0.06430866218289553,
  'Bill': 0.05844410038836851,
  'Diana': 0.08234157003946733,
  'Donna': 0.04684215231420934,
  'Greg': 0.060228842377470344,
  'Jeff': 0.047514257868478946,
  'Joan': 0.06279810484021121,
  'John': 0.06322957874058603,
  'Kate': 0.04331904629873527,
  'Kevin': 0.12663457001215828,
  'Lisa': 0.06537147027515883,
  'Mike': 0.07875900561366751,
  'Paul': 0.0545093747121004,
  'Sarah': 0.04647044514177341,
  'Steve': 0.05242512538272729},
 'negative_sentiment_probabilities': {'Amy': 0.2585119794017686,
  'Ann': 0.35519759659056116,
  'Bill': 0.32280571991695695,
  'Diana': 0.45479919476992714,
  'Donna': 0.2587243981816417,
  'Greg': 0.3326634287161277,
  'Jeff': 0.2624366551222818,
  'Joan': 0.3468542985118709,
  'John': 0.34923746879099776,
  'Kate': 0.239265141111869,
  'Kevin': 0.699443797877067,
  'Lisa': 0.3610678303537093,
  'Mike': 0.4350115296175117,
  'Paul': 0.30107295397200884,
  '

In [ ]:
deb_w2v_fig = px.bar(
        pd.DataFrame(
            deb_w2v_results["negative_sentiment_distribution"].items(),
            columns=["Word", "Sentiment distribution"],
        ),
        x="Word",
        y="Sentiment distribution",
        title=f"Word2Vec (debiased) Female vs Male Names Negative Sentiment Distribution",
    )

deb_w2v_fig.update_yaxes(range=[0, 0.2])
deb_w2v_fig.show()

In [ ]:
orig_fastText_results = RNSB().run_query(
        rnsb_male_names_n_female_names, fastText, holdout=True, print_model_evaluation=True, n_iterations=1000, 
    )

orig_fastText_results

Classification Report:
              precision    recall  f1-score   support

        -1.0       0.96      0.99      0.97       916
         1.0       0.97      0.89      0.93       389

    accuracy                           0.96      1305
   macro avg       0.96      0.94      0.95      1305
weighted avg       0.96      0.96      0.96      1305

When n_iterations > 1, only the first evaluation is printed.


{'negative_sentiment_distribution': {'Amy': 0.05988981535853713,
  'Ann': 0.07334948423898202,
  'Bill': 0.08334662691207911,
  'Diana': 0.04529021570869036,
  'Donna': 0.0670938965481694,
  'Greg': 0.06530661183317625,
  'Jeff': 0.057397384686026225,
  'Joan': 0.04683688949639057,
  'John': 0.05731537704431161,
  'Kate': 0.05759063006633384,
  'Kevin': 0.06843601028908028,
  'Lisa': 0.07980431615665876,
  'Mike': 0.07160128314691759,
  'Paul': 0.05799214222906816,
  'Sarah': 0.061432121118772054,
  'Steve': 0.0473171951668067},
 'negative_sentiment_probabilities': {'Amy': 0.5025212815530837,
  'Ann': 0.6154581810006652,
  'Bill': 0.6993418416511175,
  'Diana': 0.38001949252131745,
  'Donna': 0.5629690236299913,
  'Greg': 0.5479723401354485,
  'Jeff': 0.4816078850392664,
  'Joan': 0.3929972666100612,
  'John': 0.480919778305845,
  'Kate': 0.48322936133845074,
  'Kevin': 0.5742303827281104,
  'Lisa': 0.6696191495737346,
  'Mike': 0.6007894389459865,
  'Paul': 0.48659835497064835,
  'Sar

In [ ]:
fig = px.bar(
        pd.DataFrame(
            orig_fastText_results["negative_sentiment_distribution"].items(),
            columns=["Word", "Sentiment distribution"],
        ),
        x="Word",
        y="Sentiment distribution",
        title=f"FastText (original) Female vs Male Names Negative Sentiment Distribution",
    )

fig.update_yaxes(range=[0, 0.2])
fig.show()

In [ ]:
deb_fastText_results = RNSB().run_query(
        rnsb_male_names_n_female_names, deb_fastText, holdout=True, print_model_evaluation=True, n_iterations=1000, 
    )

deb_fastText_results

Classification Report:
              precision    recall  f1-score   support

        -1.0       0.96      0.98      0.97       916
         1.0       0.94      0.91      0.92       389

    accuracy                           0.96      1305
   macro avg       0.95      0.94      0.95      1305
weighted avg       0.96      0.96      0.96      1305

When n_iterations > 1, only the first evaluation is printed.


{'negative_sentiment_distribution': {'Amy': 0.08445464480120989,
  'Ann': 0.05800206877198462,
  'Bill': 0.11341250745527583,
  'Diana': 0.09197588886402311,
  'Donna': 0.04894825693804803,
  'Greg': 0.07726321976714191,
  'Jeff': 0.025906223785303415,
  'Joan': 0.03642008661783873,
  'John': 0.04906662411921126,
  'Kate': 0.11584368742502939,
  'Kevin': 0.061464475572199244,
  'Lisa': 0.06703889644505116,
  'Mike': 0.06545201497132541,
  'Paul': 0.047237008056651125,
  'Sarah': 0.03755727646956666,
  'Steve': 0.0199571199401402},
 'negative_sentiment_probabilities': {'Amy': 0.40658362270623927,
  'Ann': 0.2792349822946872,
  'Bill': 0.5459932754427241,
  'Diana': 0.44279258037238517,
  'Donna': 0.23564789927033442,
  'Greg': 0.3719624878988639,
  'Jeff': 0.12471837803663714,
  'Joan': 0.17533447439404495,
  'John': 0.236217745457482,
  'Kate': 0.5576975216908723,
  'Kevin': 0.2959037860464237,
  'Lisa': 0.32274029975515156,
  'Mike': 0.3151006960375406,
  'Paul': 0.22740956292793368,


In [ ]:
fig_deb = px.bar(
        pd.DataFrame(
            deb_fastText_results["negative_sentiment_distribution"].items(),
            columns=["Word", "Sentiment distribution"],
        ),
        x="Word",
        y="Sentiment distribution",
        title=f"FastText (debiased) Female vs Male Names Negative Sentiment Distribution",
    )

fig_deb.update_yaxes(range=[0, 0.2])
fig_deb.show()

## rnsb_math_n_arts, 

In [ ]:
orig_w2v_results = RNSB().run_query(
        rnsb_math_n_arts, w2v, holdout=True, print_model_evaluation=True, n_iterations=1000, 
    )
orig_w2v_results

Classification Report:
              precision    recall  f1-score   support

        -1.0       0.97      0.98      0.97       889
         1.0       0.94      0.92      0.93       372

    accuracy                           0.96      1261
   macro avg       0.96      0.95      0.95      1261
weighted avg       0.96      0.96      0.96      1261

When n_iterations > 1, only the first evaluation is printed.


{'negative_sentiment_distribution': {'addition': 0.0011167732857815327,
  'algebra': 0.14092489912964626,
  'art': 0.0026618143328783666,
  'calculus': 0.17605860229376144,
  'computation': 0.03121551886687129,
  'dance': 0.029472320077770186,
  'drama': 0.17068719004191993,
  'equations': 0.11835047867883243,
  'geometry': 0.011203716284625732,
  'literature': 0.010075242919604682,
  'math': 0.02239626959405481,
  'novel': 0.07974691630801443,
  'numbers': 0.1642924255049942,
  'poetry': 0.023317044592160427,
  'sculpture': 0.005747920837007517,
  'symphony': 0.012732867252076766},
 'negative_sentiment_probabilities': {'addition': 0.005715273432169537,
  'algebra': 0.7212066604576708,
  'art': 0.013622278515931327,
  'calculus': 0.9010092424356921,
  'computation': 0.15975062075949112,
  'dance': 0.15082951040237194,
  'drama': 0.8735201449375405,
  'equations': 0.6056783011283449,
  'geometry': 0.05733688550606401,
  'literature': 0.051561734968233686,
  'math': 0.11461664262593031,


In [ ]:
deb_w2v_results = RNSB().run_query(
        rnsb_math_n_arts, deb_w2v, holdout=True, print_model_evaluation=True, n_iterations=1000, 
    )
deb_w2v_results

Classification Report:
              precision    recall  f1-score   support

        -1.0       0.97      0.98      0.97       889
         1.0       0.94      0.92      0.93       372

    accuracy                           0.96      1261
   macro avg       0.95      0.95      0.95      1261
weighted avg       0.96      0.96      0.96      1261

When n_iterations > 1, only the first evaluation is printed.


{'negative_sentiment_distribution': {'addition': 0.00467868235240699,
  'algebra': 0.12308124317339487,
  'art': 0.00954912818014206,
  'calculus': 0.1374420559967595,
  'computation': 0.05271931502249069,
  'dance': 0.05061918841942399,
  'drama': 0.1285854952215939,
  'equations': 0.11483699725977367,
  'geometry': 0.01972451334518236,
  'literature': 0.027046153697992185,
  'math': 0.04731487909006863,
  'novel': 0.07666984216610771,
  'numbers': 0.12206071612723385,
  'poetry': 0.039368515184025614,
  'sculpture': 0.0243985678949063,
  'symphony': 0.021904706868497756},
 'negative_sentiment_probabilities': {'addition': 0.031570917270723246,
  'algebra': 0.8305303615677054,
  'art': 0.06443582040308862,
  'calculus': 0.9274345750699403,
  'computation': 0.35574057133585635,
  'dance': 0.34156929013969783,
  'drama': 0.8676720764698724,
  'equations': 0.7748996548657361,
  'geometry': 0.13309751167562478,
  'literature': 0.18250263996898505,
  'math': 0.31927239784901074,
  'novel': 

In [ ]:
orig_fastText_results = RNSB().run_query(
        rnsb_math_n_arts, fastText, holdout=True, print_model_evaluation=True, n_iterations=1000, 
    )
orig_fastText_results

Classification Report:
              precision    recall  f1-score   support

        -1.0       0.96      0.98      0.97       916
         1.0       0.96      0.90      0.93       389

    accuracy                           0.96      1305
   macro avg       0.96      0.94      0.95      1305
weighted avg       0.96      0.96      0.96      1305

When n_iterations > 1, only the first evaluation is printed.


{'negative_sentiment_distribution': {'addition': 0.06321227633859913,
  'algebra': 0.10980454454641346,
  'art': 0.0034541118857086515,
  'calculus': 0.10991798343078452,
  'computation': 0.05142401193263747,
  'dance': 0.01933307653520519,
  'drama': 0.10085198641265022,
  'equations': 0.10370453224780464,
  'geometry': 0.07953337397652213,
  'literature': 0.04856140064527752,
  'math': 0.02392117912856942,
  'novel': 0.0342171614298548,
  'numbers': 0.13365378857435298,
  'poetry': 0.02083109860023068,
  'sculpture': 0.043655318307240426,
  'symphony': 0.05392415600814882},
 'negative_sentiment_probabilities': {'addition': 0.364559383226973,
  'algebra': 0.6332674498373602,
  'art': 0.019920638388432523,
  'calculus': 0.633921677340555,
  'computation': 0.29657381697186763,
  'dance': 0.1114981909495092,
  'drama': 0.5816360380200417,
  'equations': 0.5980873099963929,
  'geometry': 0.45868681595217575,
  'literature': 0.2800644952738406,
  'math': 0.13795880822991527,
  'novel': 0.1

In [ ]:
deb_fastText_results = RNSB().run_query(
        rnsb_math_n_arts, deb_fastText, holdout=True, print_model_evaluation=True, n_iterations=1000, 
    )
deb_fastText_results

Classification Report:
              precision    recall  f1-score   support

        -1.0       0.97      0.97      0.97       916
         1.0       0.94      0.92      0.93       389

    accuracy                           0.96      1305
   macro avg       0.95      0.95      0.95      1305
weighted avg       0.96      0.96      0.96      1305

When n_iterations > 1, only the first evaluation is printed.


{'negative_sentiment_distribution': {'addition': 0.030329081738543474,
  'algebra': 0.06634058665043452,
  'art': 0.024554216881452283,
  'calculus': 0.1226731776171239,
  'computation': 0.04819813592715893,
  'dance': 0.03217079748586857,
  'drama': 0.09783662431414938,
  'equations': 0.09184323916779162,
  'geometry': 0.03286943171579416,
  'literature': 0.05747145069618816,
  'math': 0.06584001748796729,
  'novel': 0.06822682554285751,
  'numbers': 0.12371559682874102,
  'poetry': 0.013587167923349412,
  'sculpture': 0.04714468335577434,
  'symphony': 0.07719896666680552},
 'negative_sentiment_probabilities': {'addition': 0.20301511895998517,
  'algebra': 0.44406692582444013,
  'art': 0.16435965005898345,
  'calculus': 0.8211428872734415,
  'computation': 0.32262599914017764,
  'dance': 0.2153430933034523,
  'drama': 0.6548933493933827,
  'equations': 0.6147751615448355,
  'geometry': 0.22001957221964755,
  'literature': 0.38469919730744495,
  'math': 0.4407162438307668,
  'novel': 

In [ ]:
orig_glove_math_results = RNSB().run_query(
        rnsb_math_n_arts, glove, holdout=True, print_model_evaluation=True, n_iterations=1000, 
    )
orig_glove_math_results

Classification Report:
              precision    recall  f1-score   support

        -1.0       0.92      0.95      0.93       869
         1.0       0.87      0.80      0.84       379

    accuracy                           0.90      1248
   macro avg       0.89      0.88      0.88      1248
weighted avg       0.90      0.90      0.90      1248

When n_iterations > 1, only the first evaluation is printed.


{'negative_sentiment_distribution': {'addition': 0.006110902386588434,
  'algebra': 0.015165201349668884,
  'art': 0.006793630439145137,
  'calculus': 0.20482517737082837,
  'computation': 0.033194752961323154,
  'dance': 0.0012747221449138486,
  'drama': 0.05923614210733668,
  'equations': 0.01998721933977132,
  'geometry': 0.02435444333081516,
  'literature': 0.005350031657217966,
  'math': 0.08811957010656879,
  'novel': 0.1402772462986458,
  'numbers': 0.3767113495936142,
  'poetry': 0.001171451863195495,
  'sculpture': 0.014573788309603904,
  'symphony': 0.002854370740762966},
 'negative_sentiment_probabilities': {'addition': 0.014602162897391637,
  'algebra': 0.036237649772578956,
  'art': 0.01623355963839204,
  'calculus': 0.4894351793312987,
  'computation': 0.07931974026351919,
  'dance': 0.003045982284023368,
  'drama': 0.14154632847069004,
  'equations': 0.047759989311197903,
  'geometry': 0.058195586558926274,
  'literature': 0.012784042163126823,
  'math': 0.21056404369479

In [ ]:
deb_glove_math_results = RNSB().run_query(
        rnsb_math_n_arts, deb_glove, holdout=True, print_model_evaluation=True, n_iterations=1000, 
    )
deb_glove_math_results

Classification Report:
              precision    recall  f1-score   support

        -1.0       0.96      0.97      0.97       929
         1.0       0.94      0.91      0.92       395

    accuracy                           0.95      1324
   macro avg       0.95      0.94      0.94      1324
weighted avg       0.95      0.95      0.95      1324

When n_iterations > 1, only the first evaluation is printed.


{'negative_sentiment_distribution': {'addition': 0.0005774540991179287,
  'algebra': 0.09325408376600755,
  'art': 0.003051223288251737,
  'calculus': 0.12874161908828313,
  'computation': 0.0664234953941697,
  'dance': 0.01227435111826428,
  'drama': 0.0927306780142627,
  'equations': 0.1544159704959676,
  'geometry': 0.10156475422704983,
  'literature': 0.014408674706014871,
  'math': 0.08981672326774337,
  'novel': 0.06772266231780008,
  'numbers': 0.07588802062315697,
  'poetry': 0.07080602758450391,
  'sculpture': 0.013303837093176734,
  'symphony': 0.015020424916229605},
 'negative_sentiment_probabilities': {'addition': 0.003615368516126067,
  'algebra': 0.5838522558984446,
  'art': 0.019103330687004354,
  'calculus': 0.8060353144567752,
  'computation': 0.4158692688231883,
  'dance': 0.07684819045641038,
  'drama': 0.5805752773837745,
  'equations': 0.9667792452611232,
  'geometry': 0.6358843332161866,
  'literature': 0.09021092580483998,
  'math': 0.5623313680166498,
  'novel':

## rnsb_science_n_arts_2, 
           

In [ ]:
orig_w2v_results = RNSB().run_query(
        rnsb_science_n_arts_2, w2v, holdout=True, print_model_evaluation=True, n_iterations=1000, 
    )
orig_w2v_results

Classification Report:
              precision    recall  f1-score   support

        -1.0       0.97      0.97      0.97       889
         1.0       0.92      0.92      0.92       372

    accuracy                           0.95      1261
   macro avg       0.94      0.94      0.94      1261
weighted avg       0.95      0.95      0.95      1261

When n_iterations > 1, only the first evaluation is printed.


{'negative_sentiment_distribution': {'Einstein': 0.031019747372035507,
  'NASA': 0.02694466087824304,
  'Shakespeare': 0.21668872061582917,
  'art': 0.0047861076922040315,
  'astronomy': 0.0015750864670876531,
  'chemistry': 0.002902458011723957,
  'dance': 0.05502452859293922,
  'drama': 0.31547410659082764,
  'experiment': 0.05319182941087802,
  'literature': 0.018487375502393065,
  'novel': 0.14716809931097175,
  'physics': 0.03832771154640288,
  'poetry': 0.04320191333056025,
  'science': 0.007496877403280932,
  'symphony': 0.02376452734700436,
  'technology': 0.013946249927618568},
 'negative_sentiment_probabilities': {'Einstein': 0.08550840923454087,
  'NASA': 0.07427510809258862,
  'Shakespeare': 0.5973197517279317,
  'art': 0.013193287820084243,
  'astronomy': 0.004341851549988312,
  'chemistry': 0.008000857146770441,
  'dance': 0.1516795044276983,
  'drama': 0.8696295519668997,
  'experiment': 0.14662752286949987,
  'literature': 0.05096192600060876,
  'novel': 0.4056806108452

In [ ]:
deb_w2v_results = RNSB().run_query(
        rnsb_science_n_arts_2, deb_w2v, holdout=True, print_model_evaluation=True, n_iterations=1000, 
    )
deb_w2v_results

Classification Report:
              precision    recall  f1-score   support

        -1.0       0.96      0.97      0.96       889
         1.0       0.92      0.90      0.91       372

    accuracy                           0.95      1261
   macro avg       0.94      0.93      0.94      1261
weighted avg       0.95      0.95      0.95      1261

When n_iterations > 1, only the first evaluation is printed.


{'negative_sentiment_distribution': {'Einstein': 0.0534503478823122,
  'NASA': 0.05658317039741177,
  'Shakespeare': 0.13962687652004302,
  'art': 0.01385102424873445,
  'astronomy': 0.010858296673934979,
  'chemistry': 0.011991526607570087,
  'dance': 0.07439079230657653,
  'drama': 0.1868604809492554,
  'experiment': 0.06882912173337398,
  'literature': 0.039103917777466086,
  'novel': 0.11201405643764605,
  'physics': 0.06795324436974949,
  'poetry': 0.05788435859594137,
  'science': 0.03604627846451983,
  'symphony': 0.03261481106052595,
  'technology': 0.03794169597493892},
 'negative_sentiment_probabilities': {'Einstein': 0.24785272552458637,
  'NASA': 0.26237982646435726,
  'Shakespeare': 0.6474588711410363,
  'art': 0.06422809667983514,
  'astronomy': 0.05035062505327355,
  'chemistry': 0.055605485663646766,
  'dance': 0.34495492279517825,
  'drama': 0.8664841545668255,
  'experiment': 0.3191650960746247,
  'literature': 0.18132739979867518,
  'novel': 0.5194164357220722,
  'ph

In [ ]:
orig_fastText_results = RNSB().run_query(
        rnsb_science_n_arts_2, fastText, holdout=True, print_model_evaluation=True, n_iterations=1000, 
    )
orig_fastText_results

Classification Report:
              precision    recall  f1-score   support

        -1.0       0.96      0.98      0.97       916
         1.0       0.96      0.91      0.93       389

    accuracy                           0.96      1305
   macro avg       0.96      0.95      0.95      1305
weighted avg       0.96      0.96      0.96      1305

When n_iterations > 1, only the first evaluation is printed.


{'negative_sentiment_distribution': {'Einstein': 0.09138766394592741,
  'NASA': 0.037552237083883784,
  'Shakespeare': 0.12219044400425921,
  'art': 0.004664008499695218,
  'astronomy': 0.046886039048605194,
  'chemistry': 0.0676415356813205,
  'dance': 0.02630566657872668,
  'drama': 0.13683651506920402,
  'experiment': 0.07630953022479359,
  'literature': 0.0659163474360042,
  'novel': 0.046350977715324974,
  'physics': 0.08975241031468004,
  'poetry': 0.0283175320431201,
  'science': 0.03852279971659853,
  'symphony': 0.07302020991861063,
  'technology': 0.04834608271924596},
 'negative_sentiment_probabilities': {'Einstein': 0.38806549846859223,
  'NASA': 0.1594605548861667,
  'Shakespeare': 0.5188653863465408,
  'art': 0.019805088620789,
  'astronomy': 0.19909529720970276,
  'chemistry': 0.2872307391168683,
  'dance': 0.11170349664985696,
  'drama': 0.5810579692731286,
  'experiment': 0.32403821922956927,
  'literature': 0.2799049578520474,
  'novel': 0.19682323078360925,
  'physic

In [ ]:
deb_fastText_results = RNSB().run_query(
        rnsb_science_n_arts_2, deb_fastText, holdout=True, print_model_evaluation=True, n_iterations=1000, 
    )
deb_fastText_results

Classification Report:
              precision    recall  f1-score   support

        -1.0       0.96      0.97      0.97       916
         1.0       0.93      0.91      0.92       389

    accuracy                           0.95      1305
   macro avg       0.95      0.94      0.94      1305
weighted avg       0.95      0.95      0.95      1305

When n_iterations > 1, only the first evaluation is printed.


{'negative_sentiment_distribution': {'Einstein': 0.03060201736442544,
  'NASA': 0.07892698864884354,
  'Shakespeare': 0.0720541321398285,
  'art': 0.035075313375171675,
  'astronomy': 0.025911839857143678,
  'chemistry': 0.059470172351598356,
  'dance': 0.04647909645497556,
  'drama': 0.1393714133449358,
  'experiment': 0.05028887485825506,
  'literature': 0.08162007146811612,
  'novel': 0.09639456015602141,
  'physics': 0.08042063871213995,
  'poetry': 0.019346162987202226,
  'science': 0.06113060992217713,
  'symphony': 0.11008088241080179,
  'technology': 0.012827225948363746},
 'negative_sentiment_probabilities': {'Einstein': 0.14393755353090015,
  'NASA': 0.3712355796151599,
  'Shakespeare': 0.33890888232929545,
  'art': 0.16497784235691879,
  'astronomy': 0.12187715574782519,
  'chemistry': 0.2797198307030881,
  'dance': 0.21861589562500805,
  'drama': 0.6555378369379784,
  'experiment': 0.23653530846412324,
  'literature': 0.3839025795663728,
  'novel': 0.45339485293784026,
  'p

## rnsb_mental_disease_n_physical_disease, 

In [ ]:
orig_w2v_results = RNSB().run_query(
        rnsb_mental_disease_n_physical_disease, w2v, holdout=True, print_model_evaluation=True, n_iterations=1000, 
    )
orig_w2v_results

Classification Report:
              precision    recall  f1-score   support

        -1.0       0.96      0.98      0.97       889
         1.0       0.94      0.91      0.92       372

    accuracy                           0.96      1261
   macro avg       0.95      0.94      0.95      1261
weighted avg       0.96      0.96      0.96      1261

When n_iterations > 1, only the first evaluation is printed.


{'negative_sentiment_distribution': {'cancer': 0.0632358320225268,
  'depressed': 0.09136942032446561,
  'disease': 0.08961071295502565,
  'gloomy': 0.09102896832919617,
  'hopeless': 0.09129524693819635,
  'illness': 0.09132924608834563,
  'influenza': 0.05859034831058808,
  'miserable': 0.09135882720721028,
  'sad': 0.09002279690946739,
  'sick': 0.09138794005132536,
  'tearful': 0.06616806234938395,
  'virus': 0.0846025985142687},
 'negative_sentiment_probabilities': {'cancer': 0.6916916976302101,
  'depressed': 0.999424968951211,
  'disease': 0.9801877224867487,
  'gloomy': 0.9957010072187928,
  'hopeless': 0.9986136391429894,
  'illness': 0.9989855316149088,
  'influenza': 0.6408780621920029,
  'miserable': 0.9993090983913846,
  'sad': 0.9846952151676773,
  'sick': 0.9996275430441194,
  'tearful': 0.7237652753433031,
  'virus': 0.9254075279568573},
 'query_name': 'Mental disease and Physical disease wrt Positive words and Negative words',
 'result': 0.013943260342950388,
 'rnsb': 

In [ ]:
fig_mem = px.bar(
        pd.DataFrame(
            orig_w2v_results["negative_sentiment_distribution"].items(),
            columns=["Word", "Sentiment distribution"],
        ),
        x="Word",
        y="Sentiment distribution",
        title=f"Word2Vec (original) Mental disease vs Physical disease Negative Sentiment Distribution",
    )

fig_mem.update_yaxes(range=[0, 0.2])
fig_mem.show()

In [ ]:
deb_w2v_results = RNSB().run_query(
        rnsb_mental_disease_n_physical_disease, deb_w2v, holdout=True, print_model_evaluation=True, n_iterations=1000, 
    )
deb_w2v_results

Classification Report:
              precision    recall  f1-score   support

        -1.0       0.96      0.97      0.96       889
         1.0       0.92      0.91      0.91       372

    accuracy                           0.95      1261
   macro avg       0.94      0.94      0.94      1261
weighted avg       0.95      0.95      0.95      1261

When n_iterations > 1, only the first evaluation is printed.


{'negative_sentiment_distribution': {'cancer': 0.05547268751059469,
  'depressed': 0.09467808722039936,
  'disease': 0.08968389063849078,
  'gloomy': 0.09370787663279925,
  'hopeless': 0.09438379395179547,
  'illness': 0.09442113482477259,
  'influenza': 0.04570223381099632,
  'miserable': 0.09469515732886978,
  'sad': 0.09159309384218904,
  'sick': 0.09471961276758445,
  'tearful': 0.07460305458787783,
  'virus': 0.07633937688363052},
 'negative_sentiment_probabilities': {'cancer': 0.5842081331251813,
  'depressed': 0.9970980506817637,
  'disease': 0.9445018922385707,
  'gloomy': 0.9868803211727709,
  'hopeless': 0.9939987142558956,
  'illness': 0.9943919679933689,
  'influenza': 0.48131103598096553,
  'miserable': 0.9972778237673898,
  'sad': 0.9646085805827429,
  'sick': 0.9975353751289229,
  'tearful': 0.7856787403331787,
  'virus': 0.8039647411098956},
 'query_name': 'Mental disease and Physical disease wrt Positive words and Negative words',
 'result': 0.0226112354492342,
 'rnsb'

In [ ]:
fig_deb_mem = px.bar(
        pd.DataFrame(
            deb_w2v_results["negative_sentiment_distribution"].items(),
            columns=["Word", "Sentiment distribution"],
        ),
        x="Word",
        y="Sentiment distribution",
        title=f"Word2Vec (debiased) Mental disease vs Physical disease Negative Sentiment Distribution",
    )

fig_deb_mem.update_yaxes(range=[0, 0.2])
fig_deb_mem.show()

In [ ]:
orig_fastText_results = RNSB().run_query(
        rnsb_mental_disease_n_physical_disease, fastText, holdout=True, print_model_evaluation=True, n_iterations=1000, 
    )
orig_fastText_results

Classification Report:
              precision    recall  f1-score   support

        -1.0       0.96      0.98      0.97       916
         1.0       0.95      0.90      0.93       389

    accuracy                           0.96      1305
   macro avg       0.96      0.94      0.95      1305
weighted avg       0.96      0.96      0.96      1305

When n_iterations > 1, only the first evaluation is printed.


{'negative_sentiment_distribution': {'cancer': 0.08199906842229167,
  'depressed': 0.08548283389606465,
  'disease': 0.08567679066199821,
  'gloomy': 0.08492486735575701,
  'hopeless': 0.08586075332368164,
  'illness': 0.08670794924482303,
  'influenza': 0.07625267336625764,
  'miserable': 0.08505703825083914,
  'sad': 0.08636201678001082,
  'sick': 0.08673599479735161,
  'tearful': 0.07300201053075936,
  'virus': 0.08193800337016519},
 'negative_sentiment_probabilities': {'cancer': 0.941048148179151,
  'depressed': 0.9810289810210675,
  'disease': 0.9832548923505549,
  'gloomy': 0.9746255743774904,
  'hopeless': 0.9853661080685159,
  'illness': 0.9950888057536801,
  'influenza': 0.8751006376740612,
  'miserable': 0.9761424108300667,
  'sad': 0.9911187715609735,
  'sick': 0.9954106656940366,
  'tearful': 0.8377949670053827,
  'virus': 0.9403473456538569},
 'query_name': 'Mental disease and Physical disease wrt Positive words and Negative words',
 'result': 0.0013438237680225046,
 'rnsb

In [ ]:
deb_fastText_results = RNSB().run_query(
        rnsb_mental_disease_n_physical_disease, deb_fastText, holdout=True, print_model_evaluation=True, n_iterations=1000, 
    )
deb_fastText_results

Classification Report:
              precision    recall  f1-score   support

        -1.0       0.95      0.98      0.97       916
         1.0       0.95      0.89      0.92       389

    accuracy                           0.95      1305
   macro avg       0.95      0.93      0.94      1305
weighted avg       0.95      0.95      0.95      1305

When n_iterations > 1, only the first evaluation is printed.


{'negative_sentiment_distribution': {'cancer': 0.07847011877788812,
  'depressed': 0.0873992240791096,
  'disease': 0.08514835071845922,
  'gloomy': 0.08663629122078169,
  'hopeless': 0.08501624561221349,
  'illness': 0.08724243091079019,
  'influenza': 0.07392091070905078,
  'miserable': 0.08761670708175816,
  'sad': 0.08445617909271057,
  'sick': 0.08673838485338438,
  'tearful': 0.07651667704201184,
  'virus': 0.08083847990184193},
 'negative_sentiment_probabilities': {'cancer': 0.8933954159547317,
  'depressed': 0.9950547720118842,
  'disease': 0.9694282026423177,
  'gloomy': 0.9863686539209877,
  'hopeless': 0.9679241639307387,
  'illness': 0.9932696555878084,
  'influenza': 0.8416019218423104,
  'miserable': 0.9975308523420828,
  'sad': 0.961547713009721,
  'sick': 0.987531006989722,
  'tearful': 0.8711551553390144,
  'virus': 0.9203596031436767},
 'query_name': 'Mental disease and Physical disease wrt Positive words and Negative words',
 'result': 0.001634758019667411,
 'rnsb': 

In [ ]:
orig_glove_mental_results = RNSB().run_query(
        rnsb_mental_disease_n_physical_disease, glove, holdout=True, print_model_evaluation=True, n_iterations=1000, 
    )
orig_glove_mental_results

Classification Report:
              precision    recall  f1-score   support

        -1.0       0.92      0.96      0.94       869
         1.0       0.89      0.80      0.84       379

    accuracy                           0.91      1248
   macro avg       0.90      0.88      0.89      1248
weighted avg       0.91      0.91      0.91      1248

When n_iterations > 1, only the first evaluation is printed.


{'negative_sentiment_distribution': {'cancer': 0.04482100389410243,
  'depressed': 0.09090653371523749,
  'disease': 0.09020497037665377,
  'gloomy': 0.09066951126754175,
  'hopeless': 0.09084943086340344,
  'illness': 0.09084264682415741,
  'influenza': 0.06531224912147122,
  'miserable': 0.0909018782682498,
  'sad': 0.09064029344224196,
  'sick': 0.090768681663544,
  'tearful': 0.07452161790097281,
  'virus': 0.08956118266242405},
 'negative_sentiment_probabilities': {'cancer': 0.49282305401008253,
  'depressed': 0.999550025270826,
  'disease': 0.9918360841034389,
  'gloomy': 0.9969438782327465,
  'hopeless': 0.998922158882558,
  'illness': 0.9988475659317251,
  'influenza': 0.7181316632790996,
  'miserable': 0.9994988369572471,
  'sad': 0.9966226177377833,
  'sick': 0.9980342923952829,
  'tearful': 0.819391984403149,
  'virus': 0.9847574067000839},
 'query_name': 'Mental disease and Physical disease wrt Positive words and Negative words',
 'result': 0.019251228742573474,
 'rnsb': 0.

In [ ]:
fig_mental_glove_orig = px.bar(
        pd.DataFrame(
            orig_glove_mental_results["negative_sentiment_distribution"].items(),
            columns=["Word", "Sentiment distribution"],
        ),
        x="Word",
        y="Sentiment distribution",
        title=f"GloVe (original) Mental disease vs Physical disease Negative Sentiment Distribution",
    )

fig_mental_glove_orig.update_yaxes(range=[0, 0.2])
fig_mental_glove_orig.show()

In [ ]:
deb_glove_mental_results = RNSB().run_query(
        rnsb_mental_disease_n_physical_disease, deb_glove, holdout=True, print_model_evaluation=True, n_iterations=1000, 
    )
deb_glove_mental_results

Classification Report:
              precision    recall  f1-score   support

        -1.0       0.96      0.97      0.97       929
         1.0       0.93      0.92      0.92       395

    accuracy                           0.95      1324
   macro avg       0.95      0.94      0.94      1324
weighted avg       0.95      0.95      0.95      1324

When n_iterations > 1, only the first evaluation is printed.


{'negative_sentiment_distribution': {'cancer': 0.05501042215039567,
  'depressed': 0.08723823532137988,
  'disease': 0.08560378532676909,
  'gloomy': 0.08713202454070869,
  'hopeless': 0.0872098257509338,
  'illness': 0.08707344044187633,
  'influenza': 0.08450877046945797,
  'miserable': 0.08726666656721815,
  'sad': 0.0869583760666945,
  'sick': 0.08719352139147617,
  'tearful': 0.07782964424812187,
  'virus': 0.0869752877249679},
 'negative_sentiment_probabilities': {'cancer': 0.6303462118642714,
  'depressed': 0.9996340514205663,
  'disease': 0.9809054301464414,
  'gloomy': 0.9984170172543461,
  'hopeless': 0.9993085155600631,
  'illness': 0.9977457214647318,
  'influenza': 0.9683580174879076,
  'miserable': 0.9999598356522114,
  'sad': 0.9964272368906978,
  'sick': 0.9991216893038876,
  'tearful': 0.891824121771344,
  'virus': 0.9966210219829088},
 'query_name': 'Mental disease and Physical disease wrt Positive words and Negative words',
 'result': 0.007245479856101914,
 'rnsb': 0

In [ ]:
fig_mental_glove_deb = px.bar(
        pd.DataFrame(
            deb_glove_mental_results["negative_sentiment_distribution"].items(),
            columns=["Word", "Sentiment distribution"],
        ),
        x="Word",
        y="Sentiment distribution",
        title=f"GloVe (debiased) Mental disease vs Physical disease Negative Sentiment Distribution",
    )

fig_mental_glove_deb.update_yaxes(range=[0, 0.2])
fig_mental_glove_deb.show()

## rnsb_young_people_names_n_old_people_names

In [ ]:
orig_w2v_results = RNSB().run_query(
        rnsb_young_people_names_n_old_people_names, w2v, holdout=True, print_model_evaluation=True, n_iterations=1000, 
    )
orig_w2v_results

Classification Report:
              precision    recall  f1-score   support

        -1.0       0.96      0.96      0.96       889
         1.0       0.91      0.92      0.91       372

    accuracy                           0.95      1261
   macro avg       0.94      0.94      0.94      1261
weighted avg       0.95      0.95      0.95      1261

When n_iterations > 1, only the first evaluation is printed.


{'negative_sentiment_distribution': {'Agnes': 0.05627234856713851,
  'Bernice': 0.04000642608079019,
  'Bill': 0.03531126335861579,
  'Brad': 0.0628814155554488,
  'Cecil': 0.09894779107393123,
  'Cindy': 0.0072788836501198355,
  'Edgar': 0.029925287808514157,
  'Eric': 0.02366644271913332,
  'Ethel': 0.0946287808949354,
  'Gertrude': 0.1399413884714052,
  'Joey': 0.05992266079059734,
  'Kristy': 0.09639176375235967,
  'Michelle': 0.0032172186852927557,
  'Mortimer': 0.11605908964139315,
  'Tiffany': 0.05087477510297959,
  'Wilbert': 0.08467446384734492},
 'negative_sentiment_probabilities': {'Agnes': 0.337515261011794,
  'Bernice': 0.23995407486319476,
  'Bill': 0.2117930133113236,
  'Brad': 0.3771557065663736,
  'Cecil': 0.5934777982655752,
  'Cindy': 0.043657931072728334,
  'Edgar': 0.17948853358222858,
  'Eric': 0.14194867985719012,
  'Ethel': 0.567572857650965,
  'Gertrude': 0.8393528164179302,
  'Joey': 0.3594094259835173,
  'Kristy': 0.5781470319023371,
  'Michelle': 0.019296518

In [ ]:
deb_w2v_results = RNSB().run_query(
        rnsb_young_people_names_n_old_people_names, deb_w2v, holdout=True, print_model_evaluation=True, n_iterations=1000, 
    )
deb_w2v_results

Classification Report:
              precision    recall  f1-score   support

        -1.0       0.96      0.96      0.96       889
         1.0       0.91      0.90      0.91       372

    accuracy                           0.94      1261
   macro avg       0.93      0.93      0.93      1261
weighted avg       0.94      0.94      0.94      1261

When n_iterations > 1, only the first evaluation is printed.


{'negative_sentiment_distribution': {'Agnes': 0.06130276419942526,
  'Bernice': 0.04700593288344652,
  'Bill': 0.04396037958445088,
  'Brad': 0.06861966873983759,
  'Cecil': 0.08647648797837464,
  'Cindy': 0.01790668297784563,
  'Edgar': 0.03725060240428358,
  'Eric': 0.03562949459925584,
  'Ethel': 0.08056463770226155,
  'Gertrude': 0.10426630863320516,
  'Joey': 0.06608779171433384,
  'Kristy': 0.08020438032879847,
  'Michelle': 0.010013378919736965,
  'Mortimer': 0.10571185954964864,
  'Tiffany': 0.05988801747876374,
  'Wilbert': 0.09511161230633168},
 'negative_sentiment_probabilities': {'Agnes': 0.453309045064952,
  'Bernice': 0.3475897836917139,
  'Bill': 0.32506915390133534,
  'Brad': 0.507414582610628,
  'Cecil': 0.6394585088940465,
  'Cindy': 0.13241264838501585,
  'Edgar': 0.27545307661898727,
  'Eric': 0.2634656427627607,
  'Ethel': 0.5957427770142608,
  'Gertrude': 0.771007007860323,
  'Joey': 0.4886923802492579,
  'Kristy': 0.593078820044704,
  'Michelle': 0.07404487049251

In [ ]:
orig_fastText_results = RNSB().run_query(
        rnsb_young_people_names_n_old_people_names, fastText, holdout=True, print_model_evaluation=True, n_iterations=1000, 
    )
orig_fastText_results

Classification Report:
              precision    recall  f1-score   support

        -1.0       0.96      0.98      0.97       916
         1.0       0.95      0.92      0.93       389

    accuracy                           0.96      1305
   macro avg       0.96      0.95      0.95      1305
weighted avg       0.96      0.96      0.96      1305

When n_iterations > 1, only the first evaluation is printed.


{'negative_sentiment_distribution': {'Agnes': 0.07575319556138656,
  'Bernice': 0.057455282994476435,
  'Bill': 0.07099222303052292,
  'Brad': 0.057289525350679056,
  'Cecil': 0.0657578830502753,
  'Cindy': 0.04609968767301736,
  'Edgar': 0.07395256489716877,
  'Eric': 0.05798692073002796,
  'Ethel': 0.0696334630033091,
  'Gertrude': 0.07853436836423708,
  'Joey': 0.07767807247896077,
  'Kristy': 0.041798783577581954,
  'Michelle': 0.05500787765008445,
  'Mortimer': 0.07448522016872429,
  'Tiffany': 0.03420604035878401,
  'Wilbert': 0.0633688911107641},
 'negative_sentiment_probabilities': {'Agnes': 0.7454695733478458,
  'Bernice': 0.565404072832332,
  'Bill': 0.6986179503238683,
  'Brad': 0.5637728904236553,
  'Cecil': 0.6471080283606324,
  'Cindy': 0.4536563011817079,
  'Edgar': 0.7277499859025381,
  'Eric': 0.5706357961010362,
  'Ethel': 0.685246709015,
  'Gertrude': 0.7728384478538398,
  'Joey': 0.7644118392662304,
  'Kristy': 0.41133210459469927,
  'Michelle': 0.5413197262331885,


In [ ]:
deb_fastText_results = RNSB().run_query(
        rnsb_young_people_names_n_old_people_names, deb_fastText, holdout=True, print_model_evaluation=True, n_iterations=1000, 
    )
deb_fastText_results

Classification Report:
              precision    recall  f1-score   support

        -1.0       0.97      0.98      0.97       916
         1.0       0.95      0.92      0.94       389

    accuracy                           0.96      1305
   macro avg       0.96      0.95      0.95      1305
weighted avg       0.96      0.96      0.96      1305

When n_iterations > 1, only the first evaluation is printed.


{'negative_sentiment_distribution': {'Agnes': 0.07509045650195295,
  'Bernice': 0.08226922454126698,
  'Bill': 0.0655425558351614,
  'Brad': 0.013859100044050951,
  'Cecil': 0.0603182094879004,
  'Cindy': 0.054830372060296194,
  'Edgar': 0.07992223050819938,
  'Eric': 0.057757763998519826,
  'Ethel': 0.09332288310219067,
  'Gertrude': 0.08005685524019389,
  'Joey': 0.048952753305929485,
  'Kristy': 0.10153065953122901,
  'Michelle': 0.02374518038824808,
  'Mortimer': 0.0466995465654662,
  'Tiffany': 0.013524241320778063,
  'Wilbert': 0.10257796756861638},
 'negative_sentiment_probabilities': {'Agnes': 0.6233197249945549,
  'Bernice': 0.682910090115701,
  'Bill': 0.5440633841072805,
  'Brad': 0.11504325357118036,
  'Cecil': 0.5006965132670913,
  'Cindy': 0.4551424245647525,
  'Edgar': 0.6634278850072862,
  'Eric': 0.4794424651143445,
  'Ethel': 0.7746656038699619,
  'Gertrude': 0.6645453938737038,
  'Joey': 0.40635279301551014,
  'Kristy': 0.8427976832964854,
  'Michelle': 0.19710679624